# Wine quality analysis with decision trees

The file `wine_quality.csv` contains information about chemical properties of some wines. Let's see if what we learned so far can help us to predict if a wine will be good based on its properties.

## Load, examine, clean, prepare

In [ ]:
# Read and parse the wine_quality.csv file.

# The file is in CSV format. The pandas library is well
# suited to read and parse each field.
import pandas
data = pandas.read_csv("wine_quality.csv")

# We can take a look at the dataset to see what it contains.
data.head()

In [ ]:
# How many rows and columns does the dataset have ?

n_rows, n_cols = data.shape
print("The dataset has {} rows and {} columns.".format(
    n_rows, n_cols))

In [ ]:
# List all chemical properties of this dataset.

print("The chemical properties of each wine are:")
for col_name in data.columns:
    print("  -", col_name)

In [ ]:
# What kind of wines are present in this dataset ?

# With the previous question, we can see that the column
# `type` indicates what are the different kind of wines.
print("The different kind of wines in this dataset are:")
for kind in data.type.unique():
    print("  -", kind)

In [ ]:
# Find the right method to get the average/minimum/maximum value
# of each column (and only these 3 information per column)

# The method describe() of the DataFrame object `data` gives
# many information (including the average/minimum/maximum)
# but also other information like min/max or 1st/3rd quartile.
# We need to select only the interesting information (row).
data.describe().loc[["mean", "min", "max"]]

In [ ]:
# Does this dataset have any missing information ?

data.isna().head(20)
# Yes, this dataset has some missing values. For example, we
# can see a value "True" in the 17th row of the table below.

In [ ]:
# How many missing values ?

# .sum() computes the sum for each column. It gives the
# number of missing values for each column (it interprets
# True as 1 and False as 0). We need a second .sum() to get
# the number of missing values in the entire dataframe.
print("There are {} missing values in the dataframe.".
     format(data.isna().sum().sum()))

In [ ]:
# Which column has the most missing values ?

print("The column with the most missing values is '{}'.".
     format(data.isna().sum().idxmax()))

In [ ]:
# Remove the rows which have at least 1 missing value.

n_before = len(data)
data = data.dropna(axis='index') # this axis drops the rows

# How many rows have been removed ?

n_after = len(data)
print("{} rows have been removed because they contained at "
      "least 1 missing value.".format(n_before - n_after))

In [ ]:
# Use an histogram to see see the repartition of
# the wine quality.

import matplotlib.pyplot as plt
%matplotlib inline

quality_histogram = data.quality.value_counts().sort_index()
quality_histogram.plot(kind="bar", colormap="summer")
plt.title("Distribution of wines quality")
plt.show()

In [ ]:
# Let's consider that a wine is good if its quality is
# at least 7. Replace the values in the "quality" column
# with "good" if quality >= 7 and with "not good" otherwise.

# We use the function where() of the numpy library to 
# create a new column based on the values of another column.
import numpy as np
data.quality = np.where(data.quality >= 7, "good", "not good")

In [ ]:
# Create the input data (i.e. the properties) and the
# label (i.e. the quality of wine) and assign them
# to 2 different variables X and y. Our machine learning
# algorithm needs to have both input and output data.

# We choose our label (the output of our model) to be the 
# quality of the wine (either "good" or "not good"). The
# other columns are used as the input data.
X = data.drop(["quality"], axis=1)
y = data.quality

In [ ]:
# Separate your data into a training and a test set
# with 80% for the training set.

# 80% for the training set means that the test set has
# only 20% (0.2) of the data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                     test_size=0.2)

## Predicting wine quality with a decision tree

In [ ]:
# Is this a classification or a regression problem ?
# Import the appropriate version of DecisionTree, then
# train it with your training data.

# We need to predict either the label "good" or "not good".
# This is a binary classification problem (only 2 possible
# class). We import the classifier version of DecisionTree.
from sklearn.tree import DecisionTreeClassifier
wine_tree = DecisionTreeClassifier(random_state=0)
wine_tree.fit(X_train, y_train)

In [ ]:
# Oops, it seems that there is a problem! Indeed, most
# machine learning algorithms only work with numerical vectors.
# And our current training data still have some string values
# (like the type or the quality). We need to transform them before
# training our model.

# sklearn comes with tools to transform non-numerical values.
# In our case, we are going to use a LabelEncoder. Look at the
# documentation to learn what is does.

from sklearn.preprocessing import LabelEncoder
#?LabelEncoder

# now create two encoders: one for the `type` in X, the other
# for the `quality` in y. Use the trained encoders to transform
# X_train, X_test, y_train and y_test.

encoder_type = LabelEncoder()
encoder_quality = LabelEncoder()

encoder_type.fit(X.type)
encoder_quality.fit(y)

X_train.type = encoder_type.transform(X_train.type)
X_test.type = encoder_type.transform(X_test.type)
y_train = encoder_quality.transform(y_train)
y_test = encoder_quality.transform(y_test)

In [ ]:
# Now train again your Decision Tree.

wine_tree.fit(X_train, y_train)

In [ ]:
# What is the accuracy of your model (both on training
# and test sets) ? Do you think we are underfitting ? Overfitting ?

train_acc = wine_tree.score(X_train, y_train)
test_acc = wine_tree.score(X_test, y_test)
print("Train accuracy: {:.3f}".format(train_acc))
print("Test accuracy: {:.3f}".format(test_acc))

We are overfitting because the accuracy on the training set is perfect while it is not on the test set. It seems like the tree has learned by heart to predict the wine quality on the train dataset, but it cannot generalize well on data it has never seen (the test set).

In [ ]:
# Look at the documentation of your DecisionTree model
# and try to tune the hyperparameters: create other models
# with different values for max_depth, min_samples_split, max_features...
# Train them and evaluate their accuracy. What is the best accuracy
# you obtain?

max_depth = 5
wine_tree2 = DecisionTreeClassifier(max_depth=max_depth,
                                    random_state=0)
wine_tree2.fit(X_train, y_train)
print("With max_depth=", max_depth)
train_acc = wine_tree2.score(X_train, y_train)
test_acc = wine_tree2.score(X_test, y_test)
print("Train accuracy: {:.3f}".format(train_acc))
print("Test accuracy: {:.3f}".format(test_acc))
print()

max_depth = 15
wine_tree3 = DecisionTreeClassifier(max_depth=max_depth,
                                    random_state=0)
wine_tree3.fit(X_train, y_train)
print("With max_depth=", max_depth)
train_acc = wine_tree3.score(X_train, y_train)
test_acc = wine_tree3.score(X_test, y_test)
print("Train accuracy: {:.3f}".format(train_acc))
print("Test accuracy: {:.3f}".format(test_acc))
print()

min_samples_split = 5
wine_tree4 = DecisionTreeClassifier(min_samples_split=min_samples_split,
                                    random_state=0)
wine_tree4.fit(X_train, y_train)
print("With min_samples_split=", min_samples_split)
train_acc = wine_tree4.score(X_train, y_train)
test_acc = wine_tree4.score(X_test, y_test)
print("Train accuracy: {:.3f}".format(train_acc))
print("Test accuracy: {:.3f}".format(test_acc))

In [ ]:
# Use the feature_importances_ attribute of your best model. What are
# the three most important features to evaluate the quality of a wine?

# The feature_importances_ attribute is a numpy array where
# each value indicates the importance of the respective
# feature. But there is no mapping between a score and the
# chemical property it represents. We first build a DataFrame
# which contains both information.
z = pandas.DataFrame(wine_tree.feature_importances_,
                     index=X_train.columns,
                     columns=["Importance"])

# We can now sort by decreasing importance and take the
# first three features.
z.sort_values(by="Importance", ascending=False).head(3)

The three most important features to predict the quality of a wine are:
* alcohol
* sulphates
* residual sugar

## Predicting wine quality with random forests

We saw in the course (and in this example) that Decision Trees can easily overfit. To prevent this, we can use Random forests instead. Random forests are a collection of decision trees, where each decision tree is trained differently. The prediction of the RandomForest is then the average (or the most frequent) prediction of all the decision trees.

In [ ]:
# Use a RandomForest composed of 20 decision trees and
# train it on your data. Evaluate its accuracy. Do you see
# an improvement ?
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Train other random forest classifiers with different
# hyperparameters (n_estimators, max_features). Can you beat
# the best accuracy you obtained with a single decision tree ?